In [3]:
import wrds
import pandas as pd
from fuzzywuzzy import process, fuzz
import config

# 建立 WRDS 连接
conn = wrds.Connection(wrds_username=config.WRDS_USERNAME)

query = """
SELECT id_rssd, nm_lgl, date_start, date_end
FROM bank_all.wrds_struct_attributes_active
"""
bank_time_df = conn.raw_sql(query)
conn.close()

# 转换日期为datetime格式
bank_time_df['date_start'] = pd.to_datetime(bank_time_df['date_start'])
bank_time_df['date_end'] = pd.to_datetime(bank_time_df['date_end'], errors='coerce')

print(bank_time_df.head(10))

Loading library list...
Done
   id_rssd                                             nm_lgl date_start  \
0     37.0                             BANK OF HANCOCK COUNTY 2009-04-15   
1     73.0             UTILITY EMPLOYEES FEDERAL CREDIT UNION 2008-12-31   
2    242.0                   FIRST COMMUNITY BANK XENIA-FLORA 2012-01-01   
3    279.0                              BROADSTREET BANK, SSB 2023-12-04   
4    354.0                                   BISON STATE BANK 2021-07-01   
5    457.0                                   LOWRY STATE BANK 2012-01-01   
6    505.0                         BALLSTON SPA NATIONAL BANK 2017-10-25   
7    774.0                  AUSTIN TELCO FEDERAL CREDIT UNION 2022-10-19   
8    792.0                    MASSMUTUAL FEDERAL CREDIT UNION 2008-12-31   
9   1089.0  PLATTSBURGH CITY SCHOOL DISTRICT FEDERAL CREDI... 2008-12-31   

  date_end  
0      NaT  
1      NaT  
2      NaT  
3      NaT  
4      NaT  
5      NaT  
6      NaT  
7      NaT  
8      NaT  
9   

In [47]:
# 读取Primary Dealer数据
primary_dealer_df = pd.read_excel('../data/useless/Book2.xlsx')

# 转换日期为datetime
primary_dealer_df['Start Date'] = pd.to_datetime(primary_dealer_df['Start Date'])
primary_dealer_df['End Date'] = pd.to_datetime(primary_dealer_df['End Date'], errors='coerce')
primary_dealer_df.drop(columns='Unnamed: 3',inplace=True)
primary_dealer_df['End Date'] = primary_dealer_df['End Date'].fillna(pd.Timestamp.today())
primary_dealer_df = primary_dealer_df.iloc[:-2]

In [50]:
from fuzzywuzzy import fuzz, process

matched_results = []

# 遍历每一个Primary Dealer进行匹配
for idx, dealer_row in primary_dealer_df.iterrows():
    dealer_name = dealer_row['Primary Dealer']
    dealer_start = dealer_row['Start Date']
    dealer_end = dealer_row['End Date']
    
    # 先模糊匹配银行名称
    best_matches = process.extract(dealer_name, bank_time_df['nm_lgl'], scorer=fuzz.token_sort_ratio, limit=5)
    print(best_matches)

    found_match = False
    for match_name, score, _ in best_matches:
        candidate_rows = bank_time_df[bank_time_df['nm_lgl'] == match_name]
        
        # 再根据日期范围精准匹配
        for _, candidate_row in candidate_rows.iterrows():
            candidate_start = candidate_row['date_start']
            candidate_end = candidate_row['date_end'] if pd.notna(candidate_row['date_end']) else pd.Timestamp('today')
            
            # 检查日期区间重叠条件
            if (dealer_start <= candidate_end) and (dealer_end >= candidate_start):
                matched_results.append({
                    'Primary Dealer': dealer_name,
                    'Dealer Start Date': dealer_start,
                    'Dealer End Date': dealer_end,
                    'Matched Bank Name': match_name,
                    'RSSD_ID': candidate_row['id_rssd'],
                    'Bank Start Date': candidate_start,
                    'Bank End Date': candidate_end,
                    'Match Score': score
                })
                found_match = True
                break  # 找到匹配直接退出内层循环
        
        if found_match:
            break  # 找到匹配直接退出外层循环
    
    # 如果无匹配结果
    if not found_match:
        matched_results.append({
            'Primary Dealer': dealer_name,
            'Dealer Start Date': dealer_start,
            'Dealer End Date': dealer_end,
            'Matched Bank Name': None,
            'RSSD_ID': None,
            'Bank Start Date': None,
            'Bank End Date': None,
            'Match Score': None
        })

matched_results_df = pd.DataFrame(matched_results)

# 显示匹配的前20条记录
print(matched_results_df.head(20))


[('ABN AMRO BANK N.V.', 85, 36117), ('BANK OF ANN ARBOR', 60, 21166), ('FIRST ABU DHABI BANK USA N.V.', 59, 10482), ('MORGAN STANLEY BANK, N.A.', 57, 11380), ('BANK OF AMERICA S.A.', 57, 14835)]
[('DANA INCORPORATED', 84, 37557), ('ARBITRON INCORPORATED', 81, 36340), ('ABFS I INCORPORATED', 80, 22427), ('BRINER INCORPORATED', 80, 33744), ('AMAX INCORPORATED', 79, 13332)]
[('ANDREW & SUZANNE CO., INC.', 64, 23411), ('SWANTON AGENCY, INC.', 63, 8542), ('ABBEY CREDIT UNION, INC.', 62, 6265), ('MAUSTON BANCORP, INC.', 62, 10289), ('HARBEC PLASTICS INC.', 62, 32101)]
[('BBVA SECURITIES INC.', 94, 22243), ('BOFA SECURITIES, INC.', 94, 47603), ('BHC SECURITIES, INC', 91, 8766), ('ANZ SECURITIES, INC.', 91, 22117), ('FBF SECURITIES, INC.', 91, 27139)]
[('AMERICAN SECURITIES LLC', 87, 40048), ('BANC OF AMERICA SECURITIES ASIA LIMITED', 81, 10012), ('BANC OF AMERICA MORTGAGE SECURITIES, INC.', 81, 26460), ('KBFG SECURITIES AMERICA INC.', 81, 48996), ('MUFG SECURITIES AMERICAS INC.', 79, 19701)]


In [ ]:
matched_results_df.to_csv('../data/useless/第一步匹配.csv')


In [55]:
bank_time_df.to_csv('../data/useless/all_institution_RSSDID.csv')

In [7]:
conn.close()

In [8]:
# 展示 WRDS 数据库某张表的所有列名
import wrds

conn = wrds.Connection(wrds_username=config.WRDS_USERNAME)

# 使用describe_table方法查看指定表的详细信息，包括列名
table_info = conn.describe_table(table='wrds_struct_relationships', library='bank_all')

# 打印出表的列名
print("Table Columns:")
print(table_info)
conn.close()

Loading library list...
Done
Approximately 277731 rows in bank_all.wrds_struct_relationships.
Table Columns:
                  name  nullable            type  \
0    id_rssd_offspring      True  NUMERIC(15, 0)   
1       id_rssd_parent      True  NUMERIC(15, 0)   
2             ctrl_ind      True        SMALLINT   
3              reg_ind      True        SMALLINT   
4           equity_ind      True        SMALLINT   
5           pct_equity      True        SMALLINT   
6   pct_equity_bracket      True      VARCHAR(7)   
7    pct_equity_format      True      VARCHAR(7)   
8            pct_other      True  NUMERIC(15, 0)   
9               fc_ind      True        SMALLINT   
10     other_basis_ind      True        SMALLINT   
11             mb_cost      True  NUMERIC(15, 0)   
12     reason_row_crtd      True  NUMERIC(15, 0)   
13    reason_term_reln      True  NUMERIC(15, 0)   
14            regk_inv      True  NUMERIC(15, 0)   
15            reln_lvl      True  NUMERIC(15, 0)   
16     

In [21]:
## from RSSD ID to gvkey
import pandas as pd
import wrds

# 读取你的csv文件
primary_dealer_df = pd.read_csv('../data/useless/匹配RSSD_ID V3.csv').drop(columns=['Unnamed: 0'])
primary_dealer_df['RSSD ID'] = pd.to_numeric(primary_dealer_df['RSSD ID'])
primary_dealer_df['gvkey-V1'] = pd.to_numeric(primary_dealer_df['gvkey-V1'])
primary_dealer_df['Start Date'] = pd.to_datetime(primary_dealer_df['Start Date'])
primary_dealer_df['End Date'] = pd.to_datetime(primary_dealer_df['End Date'], errors='coerce')
primary_dealer_df['End Date'] = primary_dealer_df['End Date'].fillna(pd.Timestamp.today())

primary_dealer_df

,Primary Dealer,Start Date,End Date,RSSD ID,holding company,gvkey-V1
0,"AUBREY G. LANSTON & CO., INC.",1960-05-19,2000-04-17 00:00:00.000000,1573181.0,NaN,11687.0
1,BANKERS TRUST,1960-05-19,1989-07-07 00:00:00.000000,NaN,NaN,NaN
2,BARTOW LEEDS & CO.,1960-05-19,1962-06-14 00:00:00.000000,NaN,NaN,NaN
3,"C.F. CHILDS & CO., INC",1960-05-19,1965-06-29 00:00:00.000000,NaN,NaN,NaN
4,CHEMICAL,1960-05-19,1989-03-31 00:00:00.000000,1155420.0,NaN,15362.0
...,...,...,...,...,...,...
162,MORGAN STANLEY & CO. LLC,2011-05-31,2025-03-10 18:23:01.663441,1573239.0,NaN,1494.0
163,"BANK OF NOVA SCOTIA, NEW YORK AGENCY",2011-10-04,2025-03-10 18:23:01.663441,5783335.0,NaN,7647.0
164,BMO CAPITAL MARKETS CORP.,2011-10-04,2025-03-10 18:23:01.663441,3206401.0,NaN,NaN
165,JEFFERIES LLC,2013-03-01,2025-03-10 18:23:01.663441,2046020.0,NaN,NaN


In [24]:
primary_dealer_df.dtypes

Primary Dealer             object
Start Date         datetime64[ns]
End Date           datetime64[ns]
RSSD ID                   float64
holding company            object
gvkey-V1                  float64
dtype: object

In [29]:
import numpy as np
conn = wrds.Connection(wrds_username=config.WRDS_USERNAME)
# 查询控股关系 (子公司 -> 控股公司)
# RSSD IDs
today = pd.Timestamp.today()
rssd_ids = primary_dealer_df['RSSD ID'].dropna().astype(int).unique().tolist()

# 查询控股关系数据
query = f"""
SELECT id_rssd_offspring, id_rssd_parent, date_start, date_end, ctrl_ind
FROM bank_all.wrds_struct_relationships
WHERE id_rssd_offspring IN ({','.join(map(str, rssd_ids))})
AND ctrl_ind = 1
"""

relationship_df = conn.raw_sql(query)
conn.close()

# 日期处理（关系结束日期为空即当前仍然存在）
relationship_df['date_end'] = relationship_df['date_end'].fillna(today)
relationship_df['date_start'] = pd.to_datetime(relationship_df['date_start'])
relationship_df['date_end'] = pd.to_datetime(relationship_df['date_end'], errors='coerce').fillna(pd.Timestamp.today())


# 确认数据
print("从WRDS拿到的关系数据：")
print(relationship_df.head())

def safe_int_conversion(x):
    if pd.isna(x):
        return np.nan
    else:
        return int(x)
    
# 开始精确匹配
def match_holding_company(dealer_rssd, dealer_start, dealer_end):
    # 找到所有可能的控股公司
    candidates = relationship_df[relationship_df['id_rssd_offspring'] == dealer_rssd]

    # 若只有一个控股公司，无需日期过滤，直接返回
    if len(candidates) == 1:
        return candidates['id_rssd_parent'].iloc[0]

    # 若多个，需判断日期范围是否重叠
    for _, row in candidates.iterrows():
        rel_start, rel_end = row['date_start'], row['date_end']

        # 检查日期重叠逻辑
        if (dealer_start <= rel_end) and (dealer_end >= rel_start):
            return row['id_rssd_parent']

    # 若无匹配项返回NaN
    return np.nan
primary_dealer_df['RSSD ID'] = primary_dealer_df['RSSD ID'].apply(safe_int_conversion)
# 执行匹配
primary_dealer_df['Holding_Company_RSSD_ID'] = primary_dealer_df.apply(
    lambda x: match_holding_company(
        dealer_rssd=int(x['RSSD ID']),
        dealer_start=x['Start Date'],
        dealer_end=x['End Date']
    ) if pd.notna(x['RSSD ID']) else np.nan,  # NaN就返回NaN，避免转换错误
    axis=1
)

# 确认结果
print("匹配完成后的Primary Dealer数据：")
print(primary_dealer_df.head(20))


Loading library list...
Done
从WRDS拿到的关系数据：
   id_rssd_offspring  id_rssd_parent date_start   date_end  ctrl_ind
0           276579.0       1027004.0 1971-06-08 1972-04-06         1
1           276579.0       1027004.0 1972-04-07 2018-09-29         1
2           276579.0       2204549.0 1970-12-31 1971-06-07         1
3           279431.0       1020603.0 1970-12-31 1994-08-31         1
4           279431.0       1026016.0 1994-09-01 1997-06-30         1
匹配完成后的Primary Dealer数据：
                          Primary Dealer Start Date   End Date    RSSD ID  \
0    AUBREY G. LANSTON & CO., INC.       1960-05-19 2000-04-17  1573181.0   
1   BANKERS TRUST                        1960-05-19 1989-07-07        NaN   
2   BARTOW LEEDS & CO.                   1960-05-19 1962-06-14        NaN   
3   C.F. CHILDS & CO., INC               1960-05-19 1965-06-29        NaN   
4    CHEMICAL                            1960-05-19 1989-03-31  1155420.0   
5   CONTINENTAL ILL.                     1960-05-19 1988-

In [32]:
primary_dealer_df.drop(columns = ['holding company'],inplace = True)

In [35]:
import pandas as pd
import wrds

# 连接 WRDS
conn = wrds.Connection(wrds_username=config.WRDS_USERNAME)

# 获取所有 unique 的 Holding Company RSSD ID
holding_rssd_ids = primary_dealer_df['Holding_Company_RSSD_ID'].dropna().astype(int).unique().tolist()

# WRDS 查询控股公司名称
query = f"""
SELECT id_rssd, nm_lgl AS Holding_Company_Name
FROM bank_all.wrds_struct_attributes_active
WHERE id_rssd IN ({','.join(map(str, holding_rssd_ids))})
"""

# 执行查询
holding_company_df = conn.raw_sql(query)
conn.close()

# 合并查询结果
primary_dealer_df = primary_dealer_df.merge(
    holding_company_df,
    left_on='Holding_Company_RSSD_ID',
    right_on='id_rssd',
    how='left'
).drop(columns=['id_rssd'])

# 查看前几行
print("添加 Holding Company 名称后的数据：")
print(primary_dealer_df.head())



Loading library list...
Done
添加 Holding Company 名称后的数据：
                         Primary Dealer Start Date   End Date    RSSD ID  \
0   AUBREY G. LANSTON & CO., INC.       1960-05-19 2000-04-17  1573181.0   
1  BANKERS TRUST                        1960-05-19 1989-07-07        NaN   
2  BARTOW LEEDS & CO.                   1960-05-19 1962-06-14        NaN   
3  C.F. CHILDS & CO., INC               1960-05-19 1965-06-29        NaN   
4   CHEMICAL                            1960-05-19 1989-03-31  1155420.0   

   gvkey-V1  Holding_Company_RSSD_ID  holding_company_name  
0   11687.0                 958615.0                   NaN  
1       NaN                      NaN                   NaN  
2       NaN                      NaN                   NaN  
3       NaN                      NaN                   NaN  
4   15362.0                1039502.0  JPMORGAN CHASE & CO.  


In [41]:
import pandas as pd
import wrds

# 连接 WRDS
conn = wrds.Connection(wrds_username=config.WRDS_USERNAME)

# 获取 Holding Company RSSD ID
holding_rssd_ids = primary_dealer_df['Holding_Company_RSSD_ID'].dropna().astype(int).unique().tolist()

# **第一步：查询 permco**
query_permco = f"""
SELECT DISTINCT rssd9001 AS Holding_Company_RSSD_ID, permco
FROM bank_all.wrds_bank_crsp_link
WHERE rssd9001 IN ({','.join(map(str, holding_rssd_ids))})
"""

permco_df = conn.raw_sql(query_permco)

# **第二步：用 permco 去找 gvkey**
query_gvkey = """
SELECT DISTINCT lpermco AS permco, gvkey, linkdt, linkenddt
FROM crsp.ccmxpf_linktable
"""

gvkey_df = conn.raw_sql(query_gvkey)

conn.close()

# 处理时间列
gvkey_df['linkdt'] = pd.to_datetime(gvkey_df['linkdt'])
gvkey_df['linkenddt'] = pd.to_datetime(gvkey_df['linkenddt']).fillna(pd.Timestamp.today())

# **合并 permco 和 gvkey**
permco_gvkey_df = permco_df.merge(gvkey_df, on='permco', how='left')
permco_gvkey_df.rename(columns={'holding_company_rssd_id':'Holding_Company_RSSD_ID'},inplace=True)
# **合并回 primary_dealer_df**
primary_dealer_df = primary_dealer_df.merge(permco_gvkey_df, on='Holding_Company_RSSD_ID', how='left')

# **确认最终数据**
print("最终包含 Holding Company, GVKEY 和 PERMCO 的数据：")
print(primary_dealer_df.head())



Loading library list...
Done
最终包含 Holding Company, GVKEY 和 PERMCO 的数据：
                         Primary Dealer Start Date   End Date    RSSD ID  \
0   AUBREY G. LANSTON & CO., INC.       1960-05-19 2000-04-17  1573181.0   
1  BANKERS TRUST                        1960-05-19 1989-07-07        NaN   
2  BARTOW LEEDS & CO.                   1960-05-19 1962-06-14        NaN   
3  C.F. CHILDS & CO., INC               1960-05-19 1965-06-29        NaN   
4   CHEMICAL                            1960-05-19 1989-03-31  1155420.0   

   gvkey-V1  Holding_Company_RSSD_ID  holding_company_name   permco   gvkey  \
0   11687.0                 958615.0                   NaN      NaN     NaN   
1       NaN                      NaN                   NaN      NaN     NaN   
2       NaN                      NaN                   NaN      NaN     NaN   
3       NaN                      NaN                   NaN      NaN     NaN   
4   15362.0                1039502.0  JPMORGAN CHASE & CO.  20436.0  002968  

In [42]:
primary_dealer_df.to_csv('../data/useless/final_primary.csv')

In [43]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv("../data/useless/final_primary_V1.0.csv")

# 生成 1.1.csv：gvkey-V1 有值但 gvkey 为空的情况，填充 gvkey
df_1_1 = df.copy()
df_1_1.loc[df_1_1['gvkey'].isna() & df_1_1['gvkey-V1'].notna(), 'gvkey'] = df_1_1['gvkey-V1']
df_1_1.to_csv("../data/useless/final_primary_V1.1.csv", index=False)

# 生成 1.2.csv：gvkey 有值但 gvkey-V1 为空的情况，填充 gvkey-V1
df_1_2 = df.copy()
df_1_2.loc[df_1_2['gvkey-V1'].isna() & df_1_2['gvkey'].notna(), 'gvkey-V1'] = df_1_2['gvkey']
df_1_2.to_csv("../data/useless/final_primary_V1.2.csv", index=False)

print("CSV 文件 final_primary_V1.1.csv 和 final_primary_V1.2.csv 生成完成！")


CSV 文件 final_primary_V1.1.csv 和 final_primary_V1.2.csv 生成完成！
